In [2]:
## Imports
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
from datetime import datetime

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
def get_non_profit_df():
    """This function will first run the download_project_maslow_files() to check to
    see if the files have been downloaded to the `data/` folder.  If not,
    it will download the files.
    Then, it will read in the file and return a dataframe with the nonprofit
    """

    col_types = {
        "nonprofit_id": "Int64",
        "reporting_year": "Int64",
        "ein": "Int64",
        "businessname": "str",
        "phone": "str",
        "address1": "str",
        "address2": "str",
        "city": "str",
        "stabbrv": "str",
        "zip": "str"
      }

    df = pd.read_csv("../data/nonprofit.txt", sep = "|", dtype=col_types)

    return df
df_np = get_non_profit_df()
df_np = df_np[["nonprofit_id", "stabbrv"]]


In [4]:
def get_non_profit_text_df():
    """This function will first run the download_project_maslow_files() to check to
    see if the files have been downloaded to the `data/` folder.  If not,
    it will download the files.
    Then, it will read in the file and return a dataframe with the nonprofit_text
    """

    # download_project_maslow_files()÷

    col_types = {
        "nonprofit_text_id": "Int64",
        "reporting_year": "Int64",
        "nonprofit_id": "Int64",
        "grouptype": "str",
        "description": "str"
    }

    df = pd.read_csv("../data/nonprofit_text.txt", sep = "|", encoding="cp1252", dtype=col_types)

    return df

non_agg_df = get_non_profit_text_df()

# get rid of descriptions that are empty
filtered_df = non_agg_df[non_agg_df["description"].str.len() > 0]

df_text = filtered_df.groupby(['nonprofit_id'], as_index = False).agg({'description': ' '.join})

# filter down to TN
df_text = df_text.merge(df_np, on="nonprofit_id")
df_text = df_text[df_text["stabbrv"] == "TN"]

# filter down to aggregated descriptions greater than 10

df_text = df_text[df_text["description"].str.len() > 10]

In [5]:
len(df_text["nonprofit_id"].unique())

10096

In [6]:
df_text.shape

(10096, 3)

In [7]:
# create a list of the description
data = df_text.description.values.tolist()
np.shape(data)

(10096,)

In [8]:
# tokenize and clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
np.shape(data_words)

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(10096,)

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sports', 'program', 'for', 'youth', 'of', 'our', 'community', 'in', 'which', 'they', 'receive', 'sports', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission', 'inc']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
try:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
except Exception as e:
    os.system("python -m spacy download en")
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

print(np.shape(data_lemmatized))
print(data_lemmatized[:2])


/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-pack

(10096,)
[['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sport', 'program', 'youth', 'community', 'receive', 'sport', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball'], ['provide', 'food', 'clothng', 'furniture', 'economic', 'challenge', 'emergency', 'purpose', 'great', 'surround', 'area', 'provide', 'help', 'community', 'time', 'disaster', 'economic', 'interuption', 'sudden', 'loss', 'solid', 'rock', 'church', 'porvide', 'food', 'clothing', 'community']]


In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 2), (12, 2), (13, 1), (14, 1)]]


In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [14]:
def get_readable_topic(input_doc):
    if len(input_doc) > 1:
        doc = lda_model.id2word.doc2bow(input_doc)
        doc_topics, word_topics, phi_values = lda_model.get_document_topics(doc, per_word_topics=True)
        return lda_model.id2word[word_topics[0][0]]
    else:
        return None

index = 0
for doc in data_lemmatized:
    df_text.loc[index,"topic_result"] = get_readable_topic(doc)
    index = index + 1

# export model results into unique file
df_text.to_csv(f"{datetime.now().strftime('%Y_%m_%d_%H_%M')}_unique_count_{len(df_text['nonprofit_id'].unique())}_nonprofit_text_output.csv", index=False)

In [15]:
len(df_text["nonprofit_id"].unique())

10097

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.812018028488228

Coherence Score:  0.3859308018497808


In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aantonison/L

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.205940  0.065320       1        1  20.376838
5      0.210607  0.093013       2        1  18.560805
2      0.120731 -0.152060       3        1  12.979918
1      0.040038 -0.093257       4        1  10.752970
3      0.126131  0.051636       5        1  10.316393
7     -0.022379 -0.301756       6        1   9.652038
9     -0.036245  0.239080       7        1   8.310664
4     -0.258048 -0.049292       8        1   4.174170
0     -0.143305  0.075854       9        1   2.698852
8     -0.243471  0.071462      10        1   2.177353, topic_info=               Term         Freq        Total Category  logprob  loglift
6           provide  9583.000000  9583.000000  Default  30.0000  30.0000
95          service  4816.000000  4816.000000  Default  29.0000  29.0000
323          school  2904.000000  2904.000000  Default  28.0000  28.0000
234    organization  3358.000000  3358.000000  Default  27.0000  27.0000
388          member  2316.000000  2316.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
3115           step    59.975718    60.994877  Topic10  -4.8563   3.8102
1943          sense    59.313797    60.332934  Topic10  -4.8674   3.8100
3608  establishment    59.274224    60.293979  Topic10  -4.8680   3.8100
77         increase   167.367358   424.670992  Topic10  -3.8300   2.8959
301            find    80.316245   136.032128  Topic10  -4.5642   3.3001

[456 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1282      2  0.995455       abuse
377       5  0.996854    academic
173       1  0.458027      access
173       6  0.542068      access
1256      9  0.995328  accordance
...     ...       ...         ...
271       4  0.009855        year
271       5  0.258424        year
271       7  0.646059        year
360       3  0.998762       young
14        5  0.999593       youth

[541 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 3, 2, 4, 8, 10, 5, 1, 9])